In [ ]:
import os
os.chdir('..')
import sys
sys.path.insert(0, "/Users/ludo/Desktop/animalai/animalai/animalai_train")
sys.path.insert(1, "/Users/ludo/Desktop/animalai/animalai/animalai")

from raw_learned import Pipeline
from collections import namedtuple
from animalai.envs.arena_config import ArenaConfig
import os
margs = namedtuple('args', 'env seed arena_config num_episodes inference')
env_path = '../env/aaiagain'
ac = "configurations/curriculum4/35.yml"
ac = [ArenaConfig(ac)]
seed = 1
# ac = [ArenaConfig(f"../competition_configurations/{i}.yml") for i in x]
# print(f"Total number of arenas:{len(ac)}")
# args = margs(env=env_path, arena_config=ac, num_episodes=len(ac), inference=True)
args = margs(env=env_path, seed=seed, arena_config=ac, num_episodes=120, inference=False)
pipe = Pipeline(args)

In [ ]:
import time
start = time.time()
x = pipe.learn_run()
end = time.time()
print(f"Total time taken: {end-start}")
